# Demonstration of the LSST verification framework and SQuaSH 

**Authors**: Angelo Fausti

**Last Update**: Jun, 12 2019

This notebook shows how to use the [LSST verification framework](https://pipelines.lsst.io/py-api/lsst.verify) to create a new metric and track metric values in SQuaSH. In the example, we'll track an hypothetical metric, the *average camera body surface temperature* over time. For additional information, please refer to [LSST Verification Framework API Demonstration](https://sqr-019.lsst.io) and [The SQuaSH metrics dashboard](https://sqr-009.lsst.io/).


In [ ]:
import random
import getpass
import requests
import numpy as np
from astropy import units as u

import lsst.verify

## Metric definition

In the LSST verification framework (`lsst.verify`), a metric is a measurable quantity that can be tracked. 

Pipelines metrics are defined in the [verify_metrics](https://github.com/lsst/verify_metrics) metrics package.  For example, [verify_metrics/metrics/validate_drp.yaml](https://github.com/lsst/verify_metrics/blob/master/metrics/validate_drp.yaml) contains definitions for all the metrics measured by `validate_drp`.

In this demonstration, we will create an `example` metrics package to define our metric instead of using `verify_metrics`.

A metric has [name, description, unit, references, and tags](https://sqr-019.lsst.io/#Defining-metrics). Our metric is measured by an hypothetical `camera` package and is defined as follows:

In [ ]:
%%bash
cat example/metrics/camera.yaml

The metric name is prefixed by the package that measures the metric, in our example `camera.AvgCameraBodySurfaceTemp`. 

We create a `lsst.verify.MetricSet` object with the metric definiton: 

In [ ]:
example = lsst.verify.MetricSet.load_metrics_package("./example")

## Making measurements

For simplicify, let's generate an abitrary value for our metric:

In [ ]:
temp = np.random.normal(10, 0.1, 1000)
avg_temp = np.mean(temp) * u.deg_C

To captutre `avg_temp` as the metric value, we create a `lsst.verify.Measurement` object:


In [ ]:
avg_temp_meas = lsst.verify.Measurement('camera.AvgCameraBodySurfaceTemp', avg_temp)

## Creating a verification job

In `lsst.verify` a “verification job” represents a pipeline run that measure metrics. The metric values are packaged in a `lsst.verify.Job` object. With a `lsst.verify.Job` object, we can then analyze the metric values, save them to disk, and dispatch to SQuaSH. 

In [ ]:
job = lsst.verify.Job()

Here we insert the metric value into the verification job:

In [ ]:
job.measurements.insert(avg_temp_meas)

We might want to add metadata about the execution environment, the task configuration or any useful information to analyze the metric values. 

In [ ]:
job.meta.update({'camera_name': 'ComCam'})
job.meta.update({'number_of_ccds': 9})

We can also add the metric definition to the verification job, if needed:

In [ ]:
job.metrics.update(example)

**NOTE:** Serialization to disk is a temporary shim until a verification job *dataset* can be persisted through the Butler.


In [ ]:
job.write('example.json')

In [ ]:
%%bash 
cat example.json

## Dispatching verification jobs to SQuaSH

SQuaSH supports two execution environments our Jenkins CI and the LDF. Dispatching verification jobs to SQuaSH is currently automated in these environments. A DM developer might want to dispatch verification jobs to SQuaSH manually, from a local execution environment. This capability is not fully supported yet (see [QAWG-REC-37](https://jira.lsstcorp.org/browse/DM-18057)). However, we can demonstrate how it works using a sandbox instance of SQuaSH specially deployed for this purpose:

In [ ]:
SQUASH_API_URL = "https://squash-restful-api-sandbox.lsst.codes"

Only authenticated users can dispatch verification jobs to SQuaSH

In [ ]:
username = getpass.getuser()
password = getpass.getpass(prompt='Password for user `{}`: '.format(username))

If needed, we can use the SQuaSH `register` API to register a new user:

In [ ]:
r = requests.post('{}/register'.format(SQUASH_API_URL), json={'username': username, 'password': password})
r.json()

In [ ]:
r = requests.post('{}/auth'.format(SQUASH_API_URL), json={'username': username, 'password': password})
r.json()

Here we use the SQuaSH `metrics` API to upload the metric definition to the SQuaSH:

In [ ]:
r = requests.post('{}/metrics'.format(SQUASH_API_URL), json={'metrics': example.json}, headers={'Authorization': 'JWT {}'.format(r.json()['access_token'])})
r.json()

SQuaSH requires some additional metadata, in [DM-1807](https://jira.lsstcorp.org/browse/DM-18057) we want to make this more flexible: 

In [ ]:
job.meta.update({'packages': []})
job.meta.update({'env': {'env_name': 'jenkins'}})

Finally, we dispatch the verification job to SQuaSH:

In [ ]:
job.dispatch(api_user=username, api_password=password, api_url=SQUASH_API_URL)

## Analyzing metrics in Chronograf

When a verification job is dispatched to SQuaSH, the metric values and the associated metadata are stored in InfluxDB, a time-series database. [Chronograf](https://chronograf-demo.lsst.codes) is the user interface for querying and visualizing InfluxDB time-series data.

**NOTE**: The verification jobs sent to the SQuaSH sandbox instance are stored in the `squash-sandbox` InfluxDB database.


![Chronograf UI](chronograf-ui.png "Title")


## What's next

 1. Add specifications
 2. Create alert rules using Kapacitor